In [1]:
import numpy as np
import os
import pandas as pd
import random
from time import time
from rpy2.robjects import r, pandas2ri
from rpy2.robjects.packages import importr

### 1. Create the sample dataset with reduced number of variables (510)

In [2]:
# # Use a smaller data set to save time
# df = pd.read_csv('PHBsample14_sss.csv', low_memory=False)

# # drop the unnecessary columns due to csv import
# df.drop('Unnamed: 0', axis=1, inplace=True)

In [3]:
# Load the full dataset
pandas2ri.activate()
readRDS = r['readRDS']
t0 = time()
rdf = readRDS('/data/capstone_data/ClusterData_allPHB_allvendorvars_editedcolumns.rds')
df = pandas2ri.ri2py_dataframe(rdf)
t1 = time()

/home/capsops/.conda/envs/capsenvs/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Error in (function (file, refhook = NULL)  : Interrupted

  warnings.warn(x, RRuntimeWarning)


KeyboardInterrupt: 

In [ ]:
df[8,].score_C4

In [9]:
print(df.shape)
print((t1-t0)/60)

(4732698, 1063)
21.989818000793456


In [2]:
df_vars = pd.read_csv('/home/capsops/mandy/selectedVariables.csv', encoding = 'latin1')
# df_vars.drop('Unnamed: 0', axis=1, inplace=True)

In [13]:
df_vars.shape

(1063, 5)

In [15]:
len(df_vars.loc[df_vars.Delete == 0, ].Variable.values)

510

In [16]:
columnMask = [True if x in df_vars.loc[df_vars.Delete == 0, ].Variable.values else False for x in df.columns.values]

df = df.iloc[:, columnMask]

In [17]:
df.shape

(4732698, 510)

### how was the selectedVariables determined?

1. manually deleted over 200 irrelevant features; deleted features with > 50% missing values
2. run pandas_profiling to deleted over 200 correlated features
3. delete variables with the value '-2147483648'
4. delete variable 'CEN_bg_GINIIncome' since it's constant
5. replace the value of 9992 in variable 'i24aunf1_C4' with NA
6. rerun pandas_profiling to detect correlated variables, found the following 4: 'First.Mortgage.Amount.in.1000s.of.Dollars_C3', 'Target.Home.Market.Value_C3', 'Total.Mortgage.Amounts.in.1000s.of.Dollars_C3', 'WDModelFilterIn'
7. add 'PolNum_UW' back
8. delete the features with > 50% missing values
9. delete Match1,3,4 (indicator variables -- if all C4 are NaN then Match4 is N, otherwise Y)
10. delete 'Target.Narrow.Band.Income.Indicator_C3' -- categorization of variable 'Target.Narrow.Band.Income_num'

### Codes used in the above steps:

#### Replace strange values with NA

-2147483648: appears in multiple variables 

df.replace(-2147483648, np.NaN, inplace = True)

9992: i24aunf1_C4

df.i24aunf1_C4.replace('9992.0', np.NaN, inplace = True)

#### Remove the columns with more than 50% missing values

percNA = df.isnull().sum()/df.shape[0]

columnMask = percNA > 0.5

df = df.loc[:, -columnMask]

df.head(0).to_csv('selectedVariables.csv')

### 2. replace the -2147483648 value with NaN

In [18]:
df.replace(-2147483648, np.NaN, inplace = True)

/home/capsops/.conda/envs/capsenvs/lib/python3.6/site-packages/pandas/core/missing.py:50: RuntimeWarning: invalid value encountered in equal
  mask = arr == x


### 3. Encode the following boolean variables

In [19]:
for col in ['JointInd', 'GMDBInd', 'EligibleInd', 'FirstEligQInd', 'UtilizationInd']:
    df.loc[:, col].replace(1.0, 1, inplace = True)
    df.loc[:, col].replace(0.0, -1, inplace = True)

/home/capsops/.conda/envs/capsenvs/lib/python3.6/site-packages/pandas/core/missing.py:50: RuntimeWarning: invalid value encountered in equal
  mask = arr == x


In [20]:
df.loc[:, 'Qual'].replace('Q', 1, inplace = True)
df.loc[:, 'Qual'].replace('N', -1, inplace = True)
df.loc[:, 'OriginalOwner_C1'].replace('Y', 1, inplace = True)
df.loc[:, 'OriginalOwner_C1'].replace('N', -1, inplace = True)

### 4. Change the values in "Percent" columns into values between 0 and 1

In [21]:
df_vars.loc[(df_vars.Delete == 0)&(df_vars.DataMunging == 'divide by 100'), ].Variable.values

array(['Housing.Units.Percent.5.Units_C3',
       'Housing.Units.Percent.Built.2000.to.2004_C3',
       'Housing.Units.Percent.Built.2005.or.Later_C3', 'i03ccpz1_C4',
       'i03ccpp1_C4', 'i03inpp1_C4', 'i12inpp1_C4', 'i03ccpp2_C4',
       'i03ccpq1_C4', 'i12ccpq1_C4', 'i03ccpq2_C4', 'i12ccpq2_C4',
       'i03ccpq3_C4', 'i12ccpq3_C4', 'i03ccpc1_C4', 'i12ccpc1_C4',
       'i03ccpc2_C4', 'i12ccpc2_C4', 'i03ccpq4_C4', 'i12ccpq4_C4',
       'i03inpq4_C4', 'i12inpq4_C4', 'i03ccpe1_C4', 'i12ccpe1_C4'],
      dtype=object)

In [22]:
for col in df_vars.loc[(df_vars.Delete == 0)&(df_vars.DataMunging == 'divide by 100'), ].Variable.values:
    df.loc[:, col] = df.loc[:, col]/100

### 5. reorder the features by type (PolicyNum + Boolean + Categorical + Positive Integer variables + Positive Numeric + Percentage Numeric + Real Value Numeric) 

In [23]:
# columnTypes = df.dtypes
# columnUniqueCounts = df.T.apply(lambda x: x.nunique(), axis=1)
# columnSummary = pd.concat((columnTypes, columnUniqueCounts), axis = 1)
# columnSummary.columns = ['columnType', 'uniqueCounts']

# columnSummary.loc[(columnSummary.columnType == 'float64') & (columnSummary.uniqueCounts < 10), ]

In [24]:
columnMask = np.concatenate(
    [
        df_vars.loc[(df_vars.Delete == 0)&(df_vars.Type == 'PK'), ].Variable.values
      , df_vars.loc[(df_vars.Delete == 0)&(df_vars.Type == 'boolean'), ].Variable.values
      , df_vars.loc[(df_vars.Delete == 0)&(df_vars.Type == 'categorical'), ].Variable.values
      , df_vars.loc[(df_vars.Delete == 0)&(df_vars.Type == 'positive integer'), ].Variable.values
      , df_vars.loc[(df_vars.Delete == 0)&(df_vars.Type == 'positive numeric'), ].Variable.values
      , df_vars.loc[(df_vars.Delete == 0)&(df_vars.Type == 'numeric between 0 and 1'), ].Variable.values
      , df_vars.loc[(df_vars.Delete == 0)&(df_vars.Type == 'real value numeric'), ].Variable.values
    ]
)

In [25]:
df = df.loc[:, columnMask]

In [26]:
[
    len(df_vars.loc[(df_vars.Delete == 0)&(df_vars.Type == 'PK'), ].Variable.values)
  , len(df_vars.loc[(df_vars.Delete == 0)&(df_vars.Type == 'boolean'), ].Variable.values)
  , len(df_vars.loc[(df_vars.Delete == 0)&(df_vars.Type == 'categorical'), ].Variable.values)
  , len(df_vars.loc[(df_vars.Delete == 0)&(df_vars.Type == 'positive integer'), ].Variable.values)
  , len(df_vars.loc[(df_vars.Delete == 0)&(df_vars.Type == 'positive numeric'), ].Variable.values)
  , len(df_vars.loc[(df_vars.Delete == 0)&(df_vars.Type == 'numeric between 0 and 1'), ].Variable.values)
  , len(df_vars.loc[(df_vars.Delete == 0)&(df_vars.Type == 'real value numeric'), ].Variable.values)
]

[1, 7, 12, 115, 52, 313, 10]

In [79]:
# PK: 1
# Boolean: 2:8
# Categorical: 9:20
# Positive Integer: 21:135
# Positive Numeric: 136:187
# Percentage Numeric: 188:500
# Real Value Numeric: 501:510

### 6. Export cleaned data to RDS file

In [45]:
df.shape

(4732698, 510)

In [46]:
df.head()

,PolNum_UW,JointInd,GMDBInd,OriginalOwner_C1,Qual,EligibleInd,FirstEligQInd,UtilizationInd,Number.of.Adults.Indicator_C3,Dist,...,i12ccsb1_C4,i12ccsu1_C4,i12ccsv1_C4,i12ccsc1_C4,i12ccsd1_C4,i12ccsc2_C4,i03cctl1_C4,i12cctl1_C4,i12mtsq1_C4,i06txsb1_C4
1,481242,-1.0,-1.0,-1,-1,NaN,NaN,NaN,S,CA,...,16.6,0.1,0.0,0.30000,0.0,-0.400000,-4.3,0.3,0.0,0.1
2,279964,-1.0,1.0,-1,-1,NaN,NaN,NaN,,IA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,256127,-1.0,-1.0,-1,-1,NaN,NaN,NaN,S,CA,...,1.0,0.3,0.5,-1.20000,0.0,0.000000,0.0,2.7,0.0,0.0
4,515315,-1.0,-1.0,-1,-1,NaN,NaN,NaN,,WH,...,6.2,0.0,0.0,-1.10000,0.0,1.308707,0.0,105.1,0.0,-0.3
5,156276,-1.0,-1.0,-1,-1,NaN,NaN,NaN,S,IA,...,-18.7,-2.0,0.0,43.27141,0.0,-0.100000,0.0,90.5,2.0,0.0


In [50]:
t0 = time()
df.to_csv("allDataCleaned.csv", compression='gzip')
t1 = time()

NameError: name 'paste0' is not defined

In [53]:
print((t1-t0)/60/60)

3.1714428997702067
